In [1]:
from OddoDownload import OdooDownloadCenco,OdooDownloadCorona,OdooDownloadTottus
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
conn_params_cenco = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}

conn_params_corona = {
    'ODOO_USERNAME' : 'andres.romo@sinergiaindustrias.cl',
    'ODOO_PASSWORD' : 'clavefalsa',
    'ODOO_HOSTNAME' : 'repcorona-test.odoo.com',
    'ODOO_DATABASE' : 'repcorona-test'
}

conn_params_tottus = {
    'ODOO_USERNAME' : 'andres.romo@sinergiaindustrias.cl',
    'ODOO_PASSWORD' : 'clavefalsa',
    'ODOO_HOSTNAME' : 'rep-tottus-test.odoo.com',
    'ODOO_DATABASE' : 'rep-tottus-test'
}

odoo_cenco = OdooDownloadCenco(conn_params_cenco)
odoo_corona = OdooDownloadCorona(conn_params_corona)
odoo_tottus = OdooDownloadTottus(conn_params_tottus)

In [102]:
periodo = 2023

# =========================
# DESCARGAR TABLA DE VENTAS
# =========================

modelo = 'x_unidades_vendidas'
filtros = [('x_studio_periodo.x_name','=',periodo)]
campos = ['x_studio_producto','x_studio_unidades_vendidas']
header = ['Producto','Unidades vendidas'] 
campos_fk = ['x_studio_producto']

un_vendidas = odoo_tottus.getDataFromModel(modelo,filtros,campos,header,campos_fk=campos_fk,ret_=True)
un_vendidas.head()

Descargando data desde x_unidades_vendidas
.
done


,Producto,Unidades vendidas
0,400015,6922
1,1004061,9082
2,1010342,3016
3,1010343,2106
4,1010344,1949


In [103]:
# =========================
# DESCARGAR PRODUCTOS
# =========================

prods_vendidos =  un_vendidas['Producto'].to_list()

modelo = 'x_productos'
filtros = [('x_name','in',prods_vendidos)]
campos = ['x_name','x_studio_todos_los_elementos']
header = ['SKU','lista elementos'] 
campos_fk = []

productos = odoo_tottus.getDataFromModel(modelo,filtros,campos,header=header,ret_=True)
productos.head()

Descargando data desde x_productos
.
done


,SKU,Descripcion,lista elementos
0,20441996,PACK 2 BOXER NO,"[280902, 280903, 280904, 280905, 280906, 280907]"
1,20455905,PACK 2 BOXER HOMBRE,"[438737, 438738, 438739, 438740, 438741, 438742]"
2,20455908,PACK 2 BOXER HOMBRE,"[438749, 438750, 438751, 438752, 438753, 438754]"
3,20455909,PACK 2 BOXER HOMBRE,"[438755, 438756, 438757, 438758, 438759, 438760]"
4,20455910,PACK 2 BOXER HOMBRE,"[438761, 438762, 438763, 438764, 438765, 438766]"


In [109]:
# =============================
# CONTAR ELEMENTOS A DESCARGAR
# ============================
total_elementos = []
for i in range(len(productos)):
    elementos = eval(productos['lista elementos'].iloc[i])
    total_elementos += elementos

# =======================================
# DESCARGAR ELEMENTOS NESDE MATERIALIDAD
# =======================================
modelo = 'x_materialidad'
filtros = [('id','in',total_elementos)]
campos = ['x_studio_descripcion','x_name','x_studio_productos_por_envase','x_studio_peso','x_studio_peso_informado','x_studio_material','x_studio_caracteristica_material',
          'x_studio_definir_otro_opcional','x_studio_caracterstica_retornable','x_studio_caracterstica_reciclable','x_studio_peligrosidad',
          'x_studio_categoria_elemento','x_studio_sub_categoria_material','x_studio_tipo_de_parte']

header = ['Descripcion','Elemento del producto','Productos por envase','Peso','Peso informado','Material','Característica del material',
          'Composición material','Característica retornable','Característica reciclable','Peligrosidad','Categoría elemento',
          'Sub-categoría material','Tipo de parte']
campos_fk = []

elementos = odoo_tottus.getDataFromModel(modelo,filtros,campos,header=header,campos_fk=campos_fk, ret_=True,drop_id=False)
elementos.head()


Descargando data desde x_materialidad
.
.
.
done


,id,Descripcion,Elemento del producto,Productos por envase,Peso,Peso informado,Material,Característica del material,Composición material,Característica retornable,Característica reciclable,Peligrosidad,Categoría elemento,Sub-categoría material,Tipo de parte
0,1465,(D)PRG REGLAS ANIMALES,HANGING CARD,1,4.0,4.0,"[18, 'Cartón']",Cartulina,False,No Retornable,Sin material reciclado,Residuo NO Peligroso,EYE Domiciliario,PAPEL Y CARTÓN,PRIMARIO
1,1466,(D)PRG REGLAS ANIMALES,Blister,1,4.0,4.0,"[5, '(3)PVC']",Rígido,False,No Retornable,Sin material reciclado,Residuo NO Peligroso,EYE Domiciliario,PLÁSTICOS,PRIMARIO
2,1467,(D)PRG REGLAS ANIMALES,BOX,320,3.75,1200.0,"[18, 'Cartón']",Corrugado,False,No Retornable,Sin material reciclado,Residuo NO Peligroso,EYE No domiciliario,PAPEL Y CARTÓN,SECUNDARIO
3,1468,(D)PRG REGLAS ANIMALES,BOX,6400,7.5,48000.0,"[18, 'Cartón']",Corrugado,False,No Retornable,Sin material reciclado,Residuo NO Peligroso,EYE No domiciliario,PAPEL Y CARTÓN,TERCIARIO
4,1469,(D)PRG REGLAS ANIMALES,adhesive tape,6400,0.0046875,30.0,"[37, 'Envases de PP que NO contienen sustancia...",Flexible,False,No Retornable,Sin material reciclado,Residuo NO Peligroso,EYE No domiciliario,PLÁSTICOS,TERCIARIO


In [123]:
header1 = ['Producto']
header2 = ['Descripcion','Elemento del producto','Productos por envase','Peso','Peso informado','Material','Característica del material',
          'Composición material','Característica retornable','Característica reciclable','Peligrosidad','Categoría elemento',
          'Sub-categoría material','Tipo de parte']
header3 = ['Unidades vendidas']
final_header = header1 + header2 + header3

n_campos = len(final_header)
declaracion_eye = np.zeros( (len(total_elementos),n_campos),dtype='object' )


archivados = []
numero_partes = 0
index_declaracion = 0
for i in tqdm(range(len(un_vendidas))):  
    try:
        producto = un_vendidas.iloc[i]['Producto']    
        lista_elementos = eval(productos[productos['SKU']==producto]['lista elementos'].iloc[0])
        
        parte1 = np.array([producto]).reshape(-1,1)                            # header1 pero escrito de otra forma
        parte3 = un_vendidas[header3].iloc[i].to_numpy().reshape(1,-1)
    except:
        archivados.append(producto)

    for elemento in lista_elementos:
        detalle_elemento = elementos[ elementos['id']==str(elemento) ]
        detalle_elemento = detalle_elemento[header2].to_numpy().reshape(1,-1)
        row_declaracion = np.concatenate([parte1,detalle_elemento,parte3],axis=1)

        declaracion_eye[index_declaracion] = row_declaracion                # ANADE EL ELEMENTO A LA TABLA FINAL
        index_declaracion += 1




  0%|          | 0/19115 [00:00<?, ?it/s]

100%|██████████| 19115/19115 [00:23<00:00, 824.67it/s]


In [125]:
len(total_elementos)

117817